<a href="https://colab.research.google.com/github/leah-apking/project3.1/blob/main/PKE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Install & Import Dependencies

In [15]:
pip install git+https://github.com/boudinfl/pke.git

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-vfufktt_
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-vfufktt_
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.2 MB/s eta 0:00:00
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160628 sha256=7dea41009e138da972a0ddbafdc3caab1b54db7ac04e511856565014993f8a5f
  Stored in directory: /tmp/pip-ephem-wheel-cache-defek61o/wheels/8c/07/29/6b35bed2aa36e33d77ff3677eb716965ece4d2e56639ad0aab
Successfully built pke


In [16]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 132.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.9 MB/s eta 0:00:00


In [17]:
!python -m spacy download en_core_web_sm

2023-10-23 12:56:08.723867: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-23 12:56:10.469131: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 81.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [18]:
import pandas as pd
import numpy as np
import pke
from transformers import pipeline

## Pull NPS Feedback Data

In [131]:
month='09'
nps_path = f"{month}.ClientNPSSurveyFeedback.csv"
nps_data = pd.read_csv(nps_path)

In [132]:
nps = nps_data[['FullName', 'LoanNumber', 'CompanyName', 'NetPromoterLabel', 'Rating',
       'Comments', 'PerformedBy']]
nps1 = nps.rename(columns={'FullName':'ClientName', 'PerformedBy':'TeamMemberName'})
nps1["Comment"] = nps1.Comments.map(lambda x: 'False' if x == '0' else 'True')

In [133]:
nps2 = nps1.loc[nps1['Comment'] == 'True']
comments = nps2[['LoanNumber', 'Comments']]
comments_df = comments.dropna()
comments.head()

,LoanNumber,Comments
1,1.223503e+09,"Senior underwriter was a pleasure, as usual"
2,1.223512e+09,I needed a last minute change early in the mor...
4,1.523068e+09,Great Love UWM
5,1.223525e+09,Thank you for the excellent team work!
6,1.223501e+09,"Jackie Berry and Christina Plante, were awesom..."


In [134]:
comments['WordCount'] = comments['Comments'].str.count(' ') + 1
nps_short = comments.loc[(comments['WordCount'] > 0) & (comments['WordCount'] < 10)]
nps_short.count()

<ipython-input-134-a644941f0a86>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['WordCount'] = comments['Comments'].str.count(' ') + 1


LoanNumber    163
Comments      163
WordCount     163
dtype: int64

In [135]:
comments['WordCount'] = comments['Comments'].str.count(' ') + 1
nps_med = comments.loc[(comments['WordCount'] >= 10) & (comments['WordCount'] <= 30)]
nps_med.count()

<ipython-input-135-840eeb0e71f6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['WordCount'] = comments['Comments'].str.count(' ') + 1


LoanNumber    158
Comments      158
WordCount     158
dtype: int64

In [173]:
comments['WordCount'] = comments['Comments'].str.count(' ') + 1
nps_long = comments.loc[(comments['WordCount'] > 30) & (comments['WordCount'] <= 400)]
nps_long.count()

<ipython-input-173-906bb5f5c8cb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['WordCount'] = comments['Comments'].str.count(' ') + 1


LoanNumber    120
Comments      120
WordCount     120
dtype: int64

## Extract Sentiment and KeyPhrases from Comments

### "Short" Comments (WordCount < 10)

In [137]:
def get_sentiment(text):

  sentiment_pipeline = pipeline(task = 'sentiment-analysis',
                     model = 'nlptown/bert-base-multilingual-uncased-sentiment',
                     tokenizer = 'nlptown/bert-base-multilingual-uncased-sentiment')

  return sentiment_pipeline(text)

def pke_extract(text):

  extractor = pke.unsupervised.TopicRank()

  extractor.load_document(input=text, language='en')

  extractor.candidate_selection()

  extractor.candidate_weighting()

  return extractor.get_n_best(n=1, stemming=False)


nps_short['Keywords'] = nps_short['Comments'].apply(pke_extract)

nps_short['Sentiment'] = nps_short['Comments'].apply(get_sentiment)

<ipython-input-137-9bfb7f8afd4e>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Keywords'] = nps_short['Comments'].apply(pke_extract)
<ipython-input-137-9bfb7f8afd4e>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Sentiment'] = nps_short['Comments'].apply(get_sentiment)


In [138]:
nps_short['Keywords'] = nps_short['Keywords'].astype(str)

def clean_keywords(keywords_str):
  keywords_list = eval(keywords_str)
  return [keyword for keyword, _ in keywords_list]

nps_short['Keyword1'] = nps_short['Keywords'].apply(clean_keywords)

<ipython-input-138-a328b7725e19>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Keywords'] = nps_short['Keywords'].astype(str)
<ipython-input-138-a328b7725e19>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Keyword1'] = nps_short['Keywords'].apply(clean_keywords)


In [139]:
nps_short['Sentiment'] = nps_short['Sentiment'].astype(str)

def clean_sentiment(sentiment_str):
  sentiment_list = eval(sentiment_str)
  return sentiment_list[0]['label'].split(' ')[0]

nps_short['Sentiment'] = nps_short['Sentiment'].apply(clean_sentiment)

<ipython-input-139-e2e120fc2df2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Sentiment'] = nps_short['Sentiment'].astype(str)
<ipython-input-139-e2e120fc2df2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Sentiment'] = nps_short['Sentiment'].apply(clean_sentiment)


In [140]:
nps_short.head()

,LoanNumber,Comments,WordCount,Keywords,Sentiment,Keyword1
1,1.223503e+09,"Senior underwriter was a pleasure, as usual",7.0,"[('pleasure', 0.39361741945785217)]",4,[pleasure]
4,1.523068e+09,Great Love UWM,3.0,"[('great love uwm', 1.0)]",5,[great love uwm]
5,1.223525e+09,Thank you for the excellent team work!,7.0,"[('excellent team work', 1.0)]",5,[excellent team work]
9,1.223393e+09,"My AE, Terrance Watts is amazing!",6.0,"[('amazing', 0.5)]",5,[amazing]
12,1.223476e+09,Great service. speedy and easy. Thank you!,7.0,"[('speedy', 0.39361741945785217)]",5,[speedy]


### Medium Comments (WordCount 10-30)

In [141]:
def pke_extract3(text):

  extractor = pke.unsupervised.TopicRank()

  extractor.load_document(input=text, language='en')

  extractor.candidate_selection()

  extractor.candidate_weighting()

  return extractor.get_n_best(n=3, stemming=False)


nps_med['Keywords'] = nps_med['Comments'].apply(pke_extract3)

nps_med['Sentiment'] = nps_med['Comments'].apply(get_sentiment)

<ipython-input-141-e23ed7879ffe>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_med['Keywords'] = nps_med['Comments'].apply(pke_extract3)
<ipython-input-141-e23ed7879ffe>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_med['Sentiment'] = nps_med['Comments'].apply(get_sentiment)


In [142]:
nps_med['Keywords'] = nps_med['Keywords'].astype(str)
nps_med['Sentiment'] = nps_med['Sentiment'].astype(str)

nps_med['Keywords'] = nps_med['Keywords'].apply(clean_keywords)
nps_med['Sentiment'] = nps_med['Sentiment'].apply(clean_sentiment)

<ipython-input-142-ce786f379d17>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_med['Keywords'] = nps_med['Keywords'].astype(str)
<ipython-input-142-ce786f379d17>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_med['Sentiment'] = nps_med['Sentiment'].astype(str)
<ipython-input-142-ce786f379d17>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

In [160]:
#split column of lists into two new columns
split = pd.DataFrame(nps_med['Keywords'].to_list(), columns = ['Keyword1', 'Keyword2', 'Keyword3'])

#join split columns back to original DataFrame
nps_med = pd.concat([nps_med, split], axis=1)

In [145]:
nps_med.head()

,LoanNumber,Comments,WordCount,Keywords,Sentiment
6,1.223501e+09,"Jackie Berry and Christina Plante, were awesom...",17.0,"[christina plante, jackie berry, awesome]",5
8,1.223432e+09,There were a couple of hiccups in UW but was a...,15.0,"[hiccups, couple, able]",3
28,1.223497e+09,"I appreciate both my AE, Cortez for helping an...",26.0,"[questions, loan submission, closing team]",4
32,1.223493e+09,"As always, the process is seamless. I know whe...",19.0,"[uwm, loan, seamless]",5
54,1.223487e+09,Our AE Demarco is extremely helpful and access...,14.0,"[uwm underwriting team, accessible, helpful]",5


### Long Comments (WordCount 30-400)

In [174]:
# def get_sentiment(text):

#   sentiment_pipeline = pipeline(task = 'sentiment-analysis',
#                      model = 'nlptown/bert-base-multilingual-uncased-sentiment',
#                      tokenizer = 'nlptown/bert-base-multilingual-uncased-sentiment')

#   return sentiment_pipeline(text)

def pke_extract5(text):

  extractor = pke.unsupervised.TopicRank()

  extractor.load_document(input=text, language='en')

  extractor.candidate_selection()

  extractor.candidate_weighting()

  return extractor.get_n_best(n=5, stemming=False)


nps_long['Keywords'] = nps_long['Comments'].apply(pke_extract5)

nps_long['Sentiment'] = nps_long['Comments'].apply(get_sentiment)

<ipython-input-174-ecd1c7f6f5a5>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_long['Keywords'] = nps_long['Comments'].apply(pke_extract5)
<ipython-input-174-ecd1c7f6f5a5>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_long['Sentiment'] = nps_long['Comments'].apply(get_sentiment)


In [175]:
nps_long['Keywords'] = nps_long['Keywords'].astype(str)
nps_long['Sentiment'] = nps_long['Sentiment'].astype(str)

nps_long['Keywords'] = nps_long['Keywords'].apply(clean_keywords)
nps_long['Sentiment'] = nps_long['Sentiment'].apply(clean_sentiment)

<ipython-input-175-3aba88d1555d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_long['Keywords'] = nps_long['Keywords'].astype(str)
<ipython-input-175-3aba88d1555d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_long['Sentiment'] = nps_long['Sentiment'].astype(str)
<ipython-input-175-3aba88d1555d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [176]:
#split column of lists into two new columns
split = pd.DataFrame(nps_long['Keywords'].to_list(), columns = ['Keyword1', 'Keyword2', 'Keyword3', 'Keyword4', 'Keyword5'])

#join split columns back to original DataFrame
nps_long = pd.concat([nps_long, split], axis=1)

## Complete DataFrame

In [179]:
concat1 = pd.concat([nps_short, nps_med])
concat2 = pd.concat([concat1, nps_long])
concat2.head()

,LoanNumber,Comments,WordCount,Keywords,Sentiment,Keyword1,Keyword2,Keyword3,Keyword4,Keyword5
1,1.223503e+09,"Senior underwriter was a pleasure, as usual",7.0,"[('pleasure', 0.39361741945785217)]",4,[pleasure],NaN,NaN,NaN,NaN
4,1.523068e+09,Great Love UWM,3.0,"[('great love uwm', 1.0)]",5,[great love uwm],NaN,NaN,NaN,NaN
5,1.223525e+09,Thank you for the excellent team work!,7.0,"[('excellent team work', 1.0)]",5,[excellent team work],NaN,NaN,NaN,NaN
9,1.223393e+09,"My AE, Terrance Watts is amazing!",6.0,"[('amazing', 0.5)]",5,[amazing],NaN,NaN,NaN,NaN
12,1.223476e+09,Great service. speedy and easy. Thank you!,7.0,"[('speedy', 0.39361741945785217)]",5,[speedy],NaN,NaN,NaN,NaN


In [184]:
merged = pd.merge(concat2, nps1, on='LoanNumber')
merged1 = merged.drop(columns={'Keywords', 'Comments_y', 'Comment', 'NetPromoterLabel'})
final_df = merged1.rename(columns={'Comments_x': 'Comments'})
final_df.head()

,LoanNumber,Comments,WordCount,Sentiment,Keyword1,Keyword2,Keyword3,Keyword4,Keyword5,ClientName,CompanyName,Rating,TeamMemberName
0,1.223503e+09,"Senior underwriter was a pleasure, as usual",7.0,4,[pleasure],NaN,NaN,NaN,NaN,Paula Houghton,Florida Pro Mortgage LLC,10.0,Chris Lazaris
1,1.523068e+09,Great Love UWM,3.0,5,[great love uwm],NaN,NaN,NaN,NaN,John Vinecent Kordich,The Turnkey Foundation Inc. B,10.0,Rogelio Goertzen
2,1.223525e+09,Thank you for the excellent team work!,7.0,5,[excellent team work],NaN,NaN,NaN,NaN,Verna Tweed Tisdale,"Chadash Lending Group, LLC, Sunrise, FL Branch",10.0,Karl King
3,1.223393e+09,"My AE, Terrance Watts is amazing!",6.0,5,[amazing],NaN,NaN,NaN,NaN,Kasey Hazel,"Concierge Mortgage Finance, LLC",10.0,Terence Watts
4,1.223476e+09,Great service. speedy and easy. Thank you!,7.0,5,[speedy],NaN,NaN,NaN,NaN,Marcia Haym Szollosy,Schaffer Mortgage Corporation,10.0,Lamin Sambou


In [185]:
from google.colab import files
final_df.to_csv('CommentsProcessed.csv', encoding = 'utf-8-sig')
files.download('CommentsProcessed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Last week

In [ ]:
sentimentlist1 = []

ranklist1 = []
phraselist1= []
# scorelist1 = []

def sentiment1(text):

  sentiment_pipeline = pipeline("sentiment-analysis")

  sentiments = sentiment_pipeline(text)

  for s, data in enumerate(sentiments):

    sentimentlist1.append(data)

def pke_extract1(text):

  extractor = pke.unsupervised.TopicRank()

  extractor.load_document(input=text, language='en')

  extractor.candidate_selection()

  extractor.candidate_weighting()

  keyphrases1 = extractor.get_n_best(n=1, stemming=False)

  for i, (candidate, score) in enumerate(keyphrases1):

    ranklist1.append(i)
    phraselist1.append(candidate)
    # scorelist1.append(score)

for n in range(len(nps_short)):
    text = nps_short.iloc[n,1]
    sentiment1(text)
    pke_extract1(text)



No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-englis

In [ ]:
# words1 = pd.DataFrame(list(zip(phraselist1, ranklist1)), columns = ['1st', 'Rank'])
# words1

,1st,Score
0,loans,0
1,support,0
2,great job,0
3,account executive,0
4,great job,0
5,speed,0
6,cheers,0
7,closing,0
8,clean,0
9,ae beau,0


## Build DataFrame

In [ ]:
df1 = pd.DataFrame(sentimentlist1)
df3 = pd.DataFrame(sentimentlist3)
df5 = pd.DataFrame(sentimentlist5)


,label,score
0,POSITIVE,0.999392
1,POSITIVE,0.999794
2,POSITIVE,0.999848
3,POSITIVE,0.999876
4,POSITIVE,0.999876
...,...,...
127,NEGATIVE,0.998263
128,NEGATIVE,0.953232
129,NEGATIVE,0.992574
130,NEGATIVE,0.988391


In [ ]:
words1 = pd.DataFrame(list(zip(phraselist, scorelist)), columns = ['1st', 'Score'])
words1

,1st,Score
0,pleasure,0.393617
1,excellent team work,1.000000
2,amazing,0.500000
3,speedy,0.393617
4,fast process,0.408888
...,...,...
103,jumbos,0.284205
104,closing day,0.500000
105,better communication,0.500000
106,clear,0.500000


In [ ]:
def divide_chunks1(l, n):

    for i in range(0, len(l), n):
        yield l[i:i + n]
n = 1

phrases = list(divide_chunks1(phraselist, n))
columns = ['1st']
keywords_df1 = pd.DataFrame(phrases,columns=columns)
keywords_df1

,1st
0,pleasure
1,great love uwm
2,excellent team work
3,amazing
4,speedy
...,...
150,clear
151,slow process
152,slow process
153,unnecessary touches


In [ ]:
def divide_chunks3(l, n):

    for i in range(0, len(l), n):
        yield l[i:i + n]
n = 3

phrases = list(divide_chunks3(phraselist, n))
columns = ['1st', '2nd', '3rd']
keywords_df3 = pd.DataFrame(phrases,columns=columns)
keywords_df3

In [ ]:
def divide_chunks5(l, n):

    for i in range(0, len(l), n):
        yield l[i:i + n]
n = 5

phrases = list(divide_chunks5(phraselist, n))
columns = ['1st', '2nd', '3rd', '4th', '5th']
keywords_df5 = pd.DataFrame(phrases,columns=columns)
keywords_df5

,1st,2nd,3rd,4th,5th
0,uwm,closing team,awesome,easy,process
1,christina plante,jackie berry,awesome,hiccups,couple
2,able,gerry nelson,kyle smith,anthony brikho,nathan smedes
3,garett hess,conditions,succinct,gentleman,prompt
4,briana kent,questions,loan submission,closing team,help
...,...,...,...,...,...
228,fault,loan,disaster,ratings,mistake
229,file,unhelpful,dismissive,complete,rate
230,email,dollars,closing,consistent uw,nice
231,surprise,enote,uwm,agent,builder


In [ ]:
# phrases2 = list(divide_chunks(phraselist2, n))

# keywords2_df = pd.DataFrame(phrases2,columns=columns)
# keywords2_df

In [ ]:
len(scorelist)

155

In [ ]:
df1 = pd.concat([sentiment_df, keywords_df], axis=1)
# df2 = pd.concat([df1, nps_comments], axis=1)
# df2.head()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   263 non-null    object 
 1   score   263 non-null    float64
 2   1st     233 non-null    object 
 3   2nd     233 non-null    object 
 4   3rd     232 non-null    object 
 5   4th     232 non-null    object 
 6   5th     232 non-null    object 
dtypes: float64(1), object(6)
memory usage: 14.5+ KB


In [ ]:
df1

,label,score,1st,2nd,3rd,4th,5th
0,POSITIVE,0.832655,uwm,closing team,awesome,easy,process
1,POSITIVE,0.999789,christina plante,jackie berry,awesome,hiccups,couple
2,POSITIVE,0.912482,able,gerry nelson,kyle smith,anthony brikho,nathan smedes
3,POSITIVE,0.999395,garett hess,conditions,succinct,gentleman,prompt
4,NEGATIVE,0.851754,briana kent,questions,loan submission,closing team,help
...,...,...,...,...,...,...,...
258,NEGATIVE,0.999366,NaN,NaN,NaN,NaN,NaN
259,NEGATIVE,0.997779,NaN,NaN,NaN,NaN,NaN
260,NEGATIVE,0.962565,NaN,NaN,NaN,NaN,NaN
261,NEGATIVE,0.998787,NaN,NaN,NaN,NaN,NaN


In [ ]:
from google.colab import files

export_path = f"{month}_NPS_Comments_Processed.csv"
comments.to_csv(export_path, encoding = 'utf-8-sig')
files.download(export_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>